<a href="https://colab.research.google.com/github/elfabry88/SIM_YAC_SUAREZ_UPSE2023/blob/main/ExplicitaPresion2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import numpy as np
import math

In [26]:
L=30000                 #Longitud total del yacimiento
dx=2000                 #espacios entre nodos
Tiempo=100000           # tiempo en días es obtenido por (numero de iteraciones * dt= numero de días)
DT=200                   # intervalo de tiempo por día
K=50                    #Permeabilidad MD
por=0.5                 #porosidad
vis=1 # centipoise      #viscosidad
Beta=0.7
CompTotal=1.2**(-6)       #ps1-1 #compresibilidad total
A=200000#ft^2           #area de yacimiento

In [27]:
Alpha=K/(vis*por*CompTotal)  #
Alpha
                        #Se introduce Eta en la ecuacion del explicito la cual incluye alfa
Eta=(Alpha*DT)/(dx**2)  #Eta es < que 0.5 para que el sistema sea estable
Eta

0.014929919999999996

In [28]:
#ESQUEMA EXPLICITO
def Presdt(P0,P1,P2,Eta):     # son las definiciones de variables de la ecuación explicita ADICIONAL ETA
  Pdt=P1+Eta*(P0-(2*P1)+P2)   # Ecuacion del Metodo explicito con Eta
  return(Pdt)

Pexplicita=np.zeros((Tiempo//DT,L//dx)) #Crear Matriz vacia que guardara los resultados de acuerdo a filas y columnas (tiempo y nodos)

In [29]:
#Condiciones iniciales
Pyac=2000                 #Presion de yacimiento
Piz=0                     #Presion frontera izquierda
Pder=1000                 #Presion frontera derecha
Pin=np.ones(L//dx)        #Crear vector donde se guardara los resultados(condiciones iniciales-1 vector (nodos))
Pres=np.zeros(L//dx)
#Pres2=np.zeros(L//dx)
Pin=Pin*Pyac
Pin[0]=Piz                #Definimos Piz como la primera celda
Pin[len(Pin)-1]=Pder      #Definimos la longitud de celda -1 ya que Python toma como el inicio a 0
Pin

array([   0., 2000., 2000., 2000., 2000., 2000., 2000., 2000., 2000.,
       2000., 2000., 2000., 2000., 2000., 1000.])

In [ ]:
t=DT                  #Tiempo que ha tarnscurrido despues del tiempo=0
Pexplicita[0]=Pin     #los valores de la columna [0] van a guardarse los valores de Pin
while t<Tiempo:       #Ciclo de tiempo, se llena fila 2 3 4 5
  for i in range(0,len(Pin)-1): #Ciclo de espacio, q llena los nodos 1 2 3 4 se pone -1 por que no se considera el borde
    if i==0:          #i es 0 en la columna donde t es DT
      Pres[i]=Pin[0]  #cuando i es 0, Pres es la presion de condicion inicial
    elif i==len(Pin)-1: #Ahora llenamos los demas i, sera la longitud -1 python resta 1
      Pres[i]=Pin[len(Pin)-1] #Asignamos a Pres el valor anterior
    else:
      Pres[i]=Presdt(Pin[i-1],Pin[i],Pin[i+1],Eta) #llamamos a la funcion para calculas las preiones futuras con los valores i-1,i,i+1 y eta
Pexplicita[t//DT]=Pres #Secuencialmente se va a ir llenando las filas en la matriz explicita
t=t+DT                #Aumentamos los DT para que se ejecute los siguientes ciclos
Pin=Pres              #El vector in va a ser el resultado que se va obteniendo

In [ ]:
Pexplicita

In [ ]:
import matplotlib.pyplot as plt

imgplot = plt.imshow(Pexplicita,aspect='auto')
plt.colorbar()

In [ ]:
#ESQUEMA IMPLICITO
Pimplicita=np.zeros((Tiempo//DT,L//dx))

Pin=np.ones(L//dx)
Pres=np.zeros(L//dx)
Pin=Pin*Pyac
Pin[0]=Piz
Pin[len(Pin)-1]=Pder
N=L//dx
Matriz_resolucion=np.zeros([N-2,N-2])

In [ ]:
for i in range(N-2): #porque siempre se resta 2 al total de la matriz la cantidad de triangulos q tendremos
  Matriz_resolucion[i,i]=1+2*Eta #I es igual a J por eso se pone [i,i] - se llena la diagonal (1,2)(2,2)(3,3)(4,4)(5,5) etc
  if i>0:
    Matriz_resolucion[i,i-1]=-Eta #Llenar los valores a la izquierda de la diagonal con -Eta
  if i<(N-3):
    Matriz_resolucion[i,i+1]=-Eta #Llenar los valores a la derecha de la diagona N-2 no tiene valores a la derecha pero en N-3 si tiene valores y se pone -eta

In [ ]:
Pcal=np.zeros([N-2]) #
Pcal=np.linalg.solve(Matriz_resolucion,Pin[1:-1])

In [ ]:
Pcal

In [ ]:
Pder

In [ ]:
Pres=np.concatenate(([Piz],Pcal,[Pder]))
Pres

In [ ]:
Pimplicita[0]=Pin
t=DT
while t<Tiempo:
  Pcal=np.linalg.solve(Matriz_resolucion,Pin[1:-1])
  Pres=np.concatenate(([Piz],Pcal,[Pder]))
  Pimplicita[t//DT]=Pres
  Pin=Pres
  t=t+DT

In [ ]:
i#mport matplotlib.pyplot as plt

#imgplot = plt.imshow(Pexplicita,aspect='auto')
#plt.colorbar()

In [ ]:
import matplotlib.pyplot as plt

imgplot = plt.imshow(Pimplicita,aspect='auto')
plt.colorbar()